In [2]:
import pandas as pd
import configparser
config = configparser.ConfigParser()
config.read("C:\\TM1py\\tm1py-samples-master\\config.ini")
from TM1py.Objects import Cube
from TM1py.Objects import Dimension, Element, ElementAttribute, Hierarchy
from TM1py.Utils import Utils
from TM1py import TM1Service

## 加载数据

In [3]:
df=pd.read_excel(r"C:\Users\User\Desktop\ZL_Payments_Data_query.xlsx",sheet_name="Sheet1",engine="openpyxl")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\Desktop\\ZL_Payments_Data_query.xlsx'

In [3]:
df

,站点,货币,平台行为,下单时间,sku,数量,产品收入,代收销售税,运费收入,代收运费税,包装收入,代收包装税,销售折扣,代收销售折扣税,代收营业税,代缴GST,市场服务税费,市场预扣税,佣金,FBA配送费
0,202108月__de1站,EUR,FBA费用-月度仓储费,2021-08-07,S-009,1,10.00,2.00,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,1.2,-0.30,-0.50,-0.80
1,202108月__de1站,EUR,FBA费用-长期仓储费,2021-08-23,S-009,1,10.00,2.00,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3,-0.30,-0.50,-0.80
2,202108月__de2站,EUR,商品销售,2021-08-01,S-011,1,22.68,4.31,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,-4.31,-4.05,-3.38


## 拆 站点

In [4]:
Settle_Account_Year=df["站点"].str.slice(0,4)

In [5]:
Settle_Account_Month=df["站点"].str.slice(5,7)

In [6]:
Country=df["站点"].str.slice(9,11).str.upper()

In [7]:
PlatForm_Number=df["站点"].str.slice(11,13)

In [8]:
df[["Settle Account Year"]]=Settle_Account_Year

In [9]:
df[["Settle Account Month"]]=Settle_Account_Month

In [10]:
df[["Country"]]=Country

In [11]:
df[["PlatForm Number"]]=PlatForm_Number

## 拆 下单时间

In [12]:
Year=df["下单时间"].str.slice(0,4)

In [13]:
Month=df["下单时间"].str.slice(5,7)

In [14]:
Day=df["下单时间"].str.slice(5,10)

In [15]:
df[["Year"]]=Year
df[["Month"]]=Month
df[["Day"]]=Day

## 删除原站点与下单时间

In [16]:
df_clean=df.drop(["站点","下单时间"],axis=1)

In [17]:
df_clean

,货币,平台行为,sku,数量,产品收入,代收销售税,运费收入,代收运费税,包装收入,代收包装税,...,市场预扣税,佣金,FBA配送费,Settle Account Year,Settle Account Month,Country,PlatForm Number,Year,Month,Day
0,EUR,FBA费用-月度仓储费,S-009,1,10.00,2.00,1.2,1.2,1.2,1.2,...,-0.30,-0.50,-0.80,2021,8月,DE,1站,2021,08,08-07
1,EUR,FBA费用-长期仓储费,S-009,1,10.00,2.00,1.3,1.3,1.3,1.3,...,-0.30,-0.50,-0.80,2021,8月,DE,1站,2021,08,08-23
2,EUR,商品销售,S-011,1,22.68,4.31,1.4,1.4,1.4,1.4,...,-4.31,-4.05,-3.38,2021,8月,DE,2站,2021,08,08-01


## Measure维度逆透视

In [18]:
df_clean_melt=pd.melt(df_clean,id_vars=["货币","平台行为","sku","Settle Account Year","Settle Account Month","Country","PlatForm Number","Year","Month","Day"],
        var_name="Measure",value_name="Value")

In [19]:
df_clean_melt

,货币,平台行为,sku,Settle Account Year,Settle Account Month,Country,PlatForm Number,Year,Month,Day,Measure,Value
0,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,数量,1.00
1,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,数量,1.00
2,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,数量,1.00
3,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,产品收入,10.00
4,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,产品收入,10.00
5,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,产品收入,22.68
6,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,代收销售税,2.00
7,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,代收销售税,2.00
8,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,代收销售税,4.31
9,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,运费收入,1.20


## 创建维度

In [21]:
Max_Row_number=df_clean_melt.shape[0]


Dim_Dict={"平台行为":"Bahavior","sku":"eProduct","Settle Account Year":"Settle Account Year",
          "Settle Account Month":"Settle Account Month","Country":"eCountry","PlatForm Number":"PlatFormNumber","Day":"eDays",
         "Measure":"eMeasure"}


for d in Dim_Dict:
    Dim_name=Dim_Dict.get(d)



    name = Dim_name

        # Connection to TM1. Needs IP, Port, Credentials, and SSL
    with TM1Service(**config["AdventureWorks"]) as tm1:
        # create elements objects
        elements = [Element(df_clean_melt[d].iloc[i], 'Numeric') for i in range(0, Max_Row_number)]
        hierarchy = Hierarchy(name=name, dimension_name=name, elements=elements)
        dimension = Dimension(name=name, hierarchies=[hierarchy])

            # Send everything to TM1
        tm1.dimensions.create(dimension)  

## 创建维度层级结构

In [99]:
Hier_Dict={"平台行为":{"Bahavior":"所有类型"},"sku":{"eProduct":"All SKU"},"Settle Account Year":{"Settle Account Year":"All Years"},
           "Settle Account Month":{"Settle Account Month":"All Months"},
          "Country":{"eCountry":"所有国家"},"PlatForm Number":{"PlatFormNumber":"所有站点"},
           "Day":{"eDays":"所有日期"},"Measure":{"eMeasure":"All Measures"}}

for l in Hier_Dict:
    Dim=Hier_Dict[l]
    d=list(Dim.keys())[0]
    c_name=Dim.get(d)
    with TM1Service(**config["AdventureWorks"]) as tm1:
        #get dimension
        dimension = tm1.dimensions.get(d)
        #get the default hierarchy of the dimension
        h = dimension.hierarchies[0]
        #插入C level
        h.add_element(c_name, "Consolidated")
        #遍历某列的所有元素，并入C level
        for i in range(0, Max_Row_number):
            child=df_clean_melt[l].iloc[i]
            h.add_edge(c_name, child, 1)
            tm1.dimensions.update(dimension)
            

  

## 创建Cube

In [103]:
with TM1Service(**config["AdventureWorks"]) as tm1:
    cube = Cube(name='Tm1Py Cube', dimensions=['Scenario', 'Version', 'Year', 'Month',
                                               "Settle Account Year","Settle Account Month","eDays","Currency",
                                               "Bahavior","eProduct","eCountry","PlatFormNumber","eMeasure"])
    tm1.cubes.create(cube)

## 装载数据

In [118]:
#增加额外维度“ACT”，“WIP”

df_clean_melt[['Senario']]='Actual'
df_clean_melt[['Version']]='WIP'

In [119]:
df_clean_melt

,货币,平台行为,sku,Settle Account Year,Settle Account Month,Country,PlatForm Number,Year,Month,Day,Measure,Value,Senario,Version
0,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,数量,1.00,Actual,WIP
1,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,数量,1.00,Actual,WIP
2,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,数量,1.00,Actual,WIP
3,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,产品收入,10.00,Actual,WIP
4,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,产品收入,10.00,Actual,WIP
5,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,产品收入,22.68,Actual,WIP
6,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,代收销售税,2.00,Actual,WIP
7,EUR,FBA费用-长期仓储费,S-009,2021,8月,DE,1站,2021,08,08-23,代收销售税,2.00,Actual,WIP
8,EUR,商品销售,S-011,2021,8月,DE,2站,2021,08,08-01,代收销售税,4.31,Actual,WIP
9,EUR,FBA费用-月度仓储费,S-009,2021,8月,DE,1站,2021,08,08-07,运费收入,1.20,Actual,WIP


### 调整dataframe列顺序与Cube维度顺序一致

In [124]:
col_name=['Senario','Version','Year','Month','Settle Account Year','Settle Account Month','Day',
          '货币','平台行为','sku','Country','PlatForm Number','Measure','Value']

df_final=df_clean_melt.reindex(columns=col_name)

In [125]:
df_final

,Senario,Version,Year,Month,Settle Account Year,Settle Account Month,Day,货币,平台行为,sku,Country,PlatForm Number,Measure,Value
0,Actual,WIP,2021,08,2021,8月,08-07,EUR,FBA费用-月度仓储费,S-009,DE,1站,数量,1.00
1,Actual,WIP,2021,08,2021,8月,08-23,EUR,FBA费用-长期仓储费,S-009,DE,1站,数量,1.00
2,Actual,WIP,2021,08,2021,8月,08-01,EUR,商品销售,S-011,DE,2站,数量,1.00
3,Actual,WIP,2021,08,2021,8月,08-07,EUR,FBA费用-月度仓储费,S-009,DE,1站,产品收入,10.00
4,Actual,WIP,2021,08,2021,8月,08-23,EUR,FBA费用-长期仓储费,S-009,DE,1站,产品收入,10.00
5,Actual,WIP,2021,08,2021,8月,08-01,EUR,商品销售,S-011,DE,2站,产品收入,22.68
6,Actual,WIP,2021,08,2021,8月,08-07,EUR,FBA费用-月度仓储费,S-009,DE,1站,代收销售税,2.00
7,Actual,WIP,2021,08,2021,8月,08-23,EUR,FBA费用-长期仓储费,S-009,DE,1站,代收销售税,2.00
8,Actual,WIP,2021,08,2021,8月,08-01,EUR,商品销售,S-011,DE,2站,代收销售税,4.31
9,Actual,WIP,2021,08,2021,8月,08-07,EUR,FBA费用-月度仓储费,S-009,DE,1站,运费收入,1.20


In [128]:
with TM1Service(**config["AdventureWorks"]) as tm1:
    data =df_final
    #写入数据
    tm1.cells.write_dataframe(
        cube_name='Tm1Py Cube',
        data=data,
        dimensions=['Scenario', 'Version', 'Year', 'Month',
                    'Settle Account Year','Settle Account Month','eDays','Currency',
                    'Bahavior','eProduct','eCountry','PlatFormNumber','eMeasure'],
        #cellincrement
        increment=True,
        #Transaction log close and open
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        #ignore Error
        use_ti=True)